In [1]:
# Load usefull libraries and helper functions
%run libs_and_utils

seed =  42
libraries loaded


In [17]:
# Dataset consists of 6 folders each containing around 5k photographs
directory = r'input'
subfolders = [ f.path for f in os.scandir(directory) if f.is_dir()]
for subfolder in subfolders:
    print(subfolder, len(os.listdir(subfolder)))

input\andesite 5000
input\gneiss 5000
input\marble 4998
input\quartzite 5000
input\rhyolite 5000
input\schist 5000


In [33]:
for subfolder in subfolders:
    print(subfolder)
    filenames = [f.path for f in os.scandir(subfolder)]
    files_to_preview = filenames[0:10]
    for image in files_to_preview:
        display(Image(image))

input\andesite


input\gneiss


input\marble


input\quartzite


input\rhyolite


input\schist


In [32]:
%matplotlib inline


from IPython.display import display, Image


In [26]:
#%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(filenames[0])
imgplot = plt.imshow(img)
plt.show()

TypeError: Image data of dtype <U24 cannot be converted to float